In [1]:
# ! pip install python-dotenv
# !pip install --upgrade langchain
# !pip install langchain-community langchain-core
from dotenv import load_dotenv, find_dotenv
import os
from  openai  import OpenAI
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
_ = load_dotenv(find_dotenv())
api_key  = os.environ["OPENAI_API_KEY"]


In [3]:
file_name = "train_mini.csv"

df = pd.read_csv(file_name)
df.shape

(204, 6)

In [4]:
df.head(2)

,dialogue_id,conversation,turn_services,intent,service_call_method,service_call_parameters
0,1_00000,"USER:I am feeling hungry so I would like to find a place to eat.\nSYSTEM:Do you have a specific which you want the eating place to be located at?\nUSER:I would like for it to be in San Jose.\nSYSTEM:Is there a specific cuisine type you enjoy, such as Mexican, Italian or something else?\nUSER:I usually like eating the American type of food.\n",['Restaurants_1'],['FindRestaurants'],FindRestaurants,"{'city': 'San Jose', 'cuisine': 'American'}"
1,1_00001,"USER:Hi i need a help, i am very hungry, I am looking for a restaurant\nSYSTEM:Sure, I will help you, What type of food are you looking for? Which city should i search in?\nUSER:Some Punjabi kind of foods in milpitas\n",['Restaurants_1'],['FindRestaurants'],FindRestaurants,"{'city': 'Milpitas', 'cuisine': 'Indian'}"


In [5]:
conversation = df.iloc[1]['conversation']
print(conversation)

USER:Hi i need a help, i am very hungry, I am looking for a restaurant
SYSTEM:Sure, I will help you, What type of food are you looking for? Which city should i search in?
USER:Some Punjabi kind of foods in milpitas



In [6]:
# !pip install -U langchain-openai

from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI

prompt_template = """
Your role is to ask an agent to {intent}. 
Construct a one line user instruction based on this intention as first person from the conversation.

Focus on whether user is asking you to {intent}. 
Also identify requiered number of people, time , place or any other available entity to {intent} in the conversation.
Do not display these entities separately though.
Use this information to construct the user instruction. Do not make up information if it is not availble.
Keep the same verb from conversation in the instruction that is used to {intent}.
Use diverse mood and personality in the instruction.
In the instruction put date/time, place and number of people for {intent} in the different order. 


The conversation is delimited with triple backticks.
Conversation: ```{conversation}```
"""

In [7]:
llm_name = "gpt-4o-mini"
llm = ChatOpenAI(model_name=llm_name, temperature=0.5)

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

C:\Users\mal-masud\AppData\Local\Temp\ipykernel_26812\1878140874.py:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  llm_chain = LLMChain(


In [8]:
# result = llm_chain.predict(conversation=conversation)
# result

### Running over dataframe      


In [9]:

results = []
intent_dict = {
    'ReserveRestaurant' : 'Reserve a Restaurant',
    'FindRestaurants':'Find a Restaurant',
    'SearchHotel': 'Search a Hotel',
    'ReserveHotel':'Reserve a Hotel'


}

# Experimentation : Iterate over each row
# for i in  range(1,205,50):
#     # Apply LLM chain on the desired column in each row
#     result = llm_chain.predict(
#         conversation=df.iloc[i]['dialogue_id']+': '+df.iloc[i]['conversation'],
#         intent=intent_dict[df.iloc[i]['service_call_method']])

#     # results.append([df.iloc[i]['conversation'],result])
#     # results.append(result)
#     print(df.iloc[i]['dialogue_id'],df.iloc[i]['service_call_method']+'\n',df.iloc[i]['conversation'])
#     print(result)
#     print('='*len(result))

# Iterate over each row
for index, row in df.iterrows():
    # Apply LLM chain on the desired column in each row
    result = llm_chain.predict(
        conversation=row['dialogue_id']+': '+row['conversation'],
        intent=intent_dict[row['service_call_method']])

    results.append(result)
    


In [10]:
df['instruction']=results

In [18]:
df_out=df[['dialogue_id','service_call_method','conversation','instruction','service_call_parameters']]

In [19]:
df_out.to_csv('instruction_mini.csv')

In [12]:
# def conversation_to_instruction(conversation):
#   MODEL="gpt-4o-mini"

#   completion = client.chat.completions.create(
#     model=MODEL,
#     messages=[
#           {"role": "system", "content": f"""Here is a conversation between a user and the system. Please draft a one sentence user instruction.
#           Please keep in mind:
#           - randomize your mood and personality
#           - randomize orders in each sentence.
#           - randomize structure of sentences
#           - each instruction must have number of people, time and place  in sentences if available
          


#           Here is the data:
#           {conversation}
#           """},
          
#       ]
#   )
#   return completion.choices[0].message.content

In [13]:
# csv_file = r'C:\openai_proj\train_50.csv'
# # Load the CSV file
# df = pd.read_csv(csv_file)
# conversation = " "

# added_instruction_count = 0
# for index,row in df.iterrows():
#     intent = row['service_call_method']
#     conversation = row['conversation']
#     if  not pd.isna(row['instruction']):
#         continue
        
    
    
#     str_instructions = conversation_to_instruction(conversation)
#     df.loc[index,'instruction'] = str_instructions
#     added_instruction_count += 1
#     if added_instruction_count > 200: break
# df = df.reindex(['dialogue_id', 'intent','conversation', 'instruction', 'services', 
#        'service_call_method', 'service_call_parameters' ],axis=1)
# df.to_csv(csv_file)
    





In [14]:
!python --version


Python 3.12.4


In [15]:
conversation

'USER:Hi i need a help, i am very hungry, I am looking for a restaurant\nSYSTEM:Sure, I will help you, What type of food are you looking for? Which city should i search in?\nUSER:Some Punjabi kind of foods in milpitas\n'

In [16]:
len(list_instructions)

NameError: name 'list_instructions' is not defined